In [4]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from time import sleep
import selenium.webdriver.common.keys
from selenium_stealth import stealth
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import config
import random

In [5]:
def rand_proxy():
    proxy = random.choice(config.ips)
    return proxy
proxy = rand_proxy()

#First uncomment bottom, then uncomment top one
#chrome_options.add_argument(f'--proxy-server={proxy}') 
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument(f'--proxy-server={proxy}')

#chrome_options.add_argument("--headless=new")
chrome_options.add_argument("start-maximized")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.page_load_strategy = 'normal'
chrome_options.binary_location = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"

driver = webdriver.Chrome(options=chrome_options)
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",)

s = Service("C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe")
new_data = []

def get_jobs():
    how_many_scrolls=0
    driver.get("https://www.linkedin.com/jobs/search?keywords=data&location=India&geoId=102713980&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")
    scroll_pause_time =2
    scroll_height=0
    #Change the value in while loop to change number of scrolls | currently set to 1 scroll
    while how_many_scrolls<1:
        prev_scroll_height = scroll_height
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {scroll_height});")
        time.sleep(scroll_pause_time)
        if scroll_height == prev_scroll_height:
            try:
                driver.find_element(By.CLASS_NAME, "infinite-scroller__show-more-button").click()
                how_many_scrolls+=1
            except:
                pass

    jobs = driver.find_elements(By.XPATH,f'//*[@id="main-content"]/section[2]/ul/li')  
    for noj in jobs:
        try:
            noj.click()
            time.sleep(2)
            job_name = (noj.find_element(By.XPATH, '/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/a/h2')).text
            com_name = (noj.find_element(By.XPATH, '/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/h4/div[1]/span[1]/a')).text
            loc = (noj.find_element(By.XPATH, '/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/h4/div[1]/span[2]')).text
            job_level = (noj.find_element(By.XPATH, '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[1]/span')).text
            job_type = (noj.find_element(By.XPATH, '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[2]/span')).text
            job_func = (noj.find_element(By.XPATH, '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[3]/span')).text
            job_ind = (noj.find_element(By.XPATH, '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[4]/span')).text
            new_data.append([job_name,com_name,loc,job_level,job_func,job_type,job_ind])
        except:
            pass

    driver.quit()
    df1 = pd.DataFrame(new_data, columns=['Job_Title', 'Company', 'Location','Level', 'Type', 'Function','Industry'])
    df1.to_csv('linkedin_Scraped_data.csv')
    
get_jobs()